# Load Data

In [2]:
using DataFrames

# To run this notebook, you need to have a data file available.
# You can either run the phenotype preprocessing scripts in ../preprocessing directory
# Or scp -r username@sherlock.stanford.edu:/scratch/PI/dpwall/DATA/phenotypes/jsonschema ../data

# Data
df = readtable("../data/all_samples_filtered.csv", nastrings=["None", ""])
samples = df[:, 1:1]
df = df[:, 2:end] # Remove identifier

# Binarize Data
[df[df[nm].> 0, nm] = 1 for nm in names(df)]
[df[df[nm].== 0, nm] = -1 for nm in names(df)]
[df[isna(df[nm]), nm] = 0 for nm in names(df)]

m, n = size(df)


(13434,123)

In [3]:
# Form sparse array
all_data = sparse(Array(df))
dropzeros!(all_data)
p = size(nonzeros(all_data), 1)

1001073

In [4]:
adir_indices = 1:77
ados_indices = 78:n

78:123

In [5]:
# First split out whole instrument test data

dropzeros!(all_data)
p = size(nonzeros(all_data), 1)

# Split out testing data
all_indices = collect(1:p)
shuffle!(all_indices)
break1, break2 = ceil(Integer, 0.85 * p), ceil(Integer, 0.9 * p)
train_indices = view(all_indices, 1:(break1-1))
test_indices = view(all_indices, break1:(break2-1))
held_out_test_indices = view(all_indices, break2:p)

train_data = copy(all_data)
nonzeros(train_data)[union(test_indices, held_out_test_indices)] = 0
dropzeros!(train_data)

test_data = copy(all_data)
nonzeros(test_data)[union(train_indices, held_out_test_indices)] = 0
dropzeros!(test_data)

heldout_data = copy(all_data)
nonzeros(heldout_data)[union(train_indices, test_indices)] = 0
dropzeros!(heldout_data)

println(size(held_out_test_indices), " ", size(nonzeros(heldout_data)))
println(size(test_indices), " ", size(nonzeros(test_data)))
println(size(train_indices), " ", size(nonzeros(train_data)))

(100108,) (100108,)
(50053,) (50053,)
(850912,) (850912,)


In [6]:
println(size(nonzeros(heldout_data[:, adir_indices])))
println(size(nonzeros(heldout_data[:, ados_indices])))

(69062,)
(31046,)


# Train Models

## Both instruments

In [11]:
using LowRankModels

Xs = []
Ys = []
for k=1:6
    #losses = [MultinomialLoss(3) for i=1:n]
    losses = LogisticLoss()
    
    rx = ZeroReg()
    ry = ZeroReg()
    glrm = GLRM(train_data, losses, rx, ry, k, offset=false, scale=false);
    #init_svd!(glrm);

    X,Y,ch = fit!(glrm, verbose=true, max_iter=5000); # fit GLRM
    push!(Xs, X)
    push!(Ys, Y)
end


LowRankModels.SparseProxGradParams(1.0,100,1,1.0e-5,0.01)
Fitting GLRM
Iteration 10: objective value = 498850.4422640237
Iteration 20: objective value = 405237.1670185198
Iteration 30: objective value = 404014.02541176596
Iteration 40: objective value = 403756.15829549683
obj went up to 403757.12108088395; reducing step size to 4.693325808083107
obj went up to 403786.8095543603; reducing step size to 3.285328065658175
obj went up to 403757.31623747176; reducing step size to 2.1902187104387836
LowRankModels.SparseProxGradParams(1.0,100,1,1.0e-5,0.01)
Fitting GLRM
Iteration 10: objective value = 540946.105836054
Iteration 20: objective value = 384239.94367045566
Iteration 30: objective value = 359772.30522771756
Iteration 40: objective value = 355261.08245209785
obj went up to 355913.98830866505; reducing step size to 4.693325808083107
obj went up to 355419.4667983449; reducing step size to 3.1288838720554044
obj went up to 355258.06689252576; reducing step size to 2.190218710438783
Iter

# Evaluate

## Both Instruments

In [10]:
Xs = []
Ys = []
error = []
adir_error = []
ados_error = []
for l=1:6
    println(l)
    approx = Xs[l].'*Ys[l]
    approx[approx.>=0] = 1
    approx[approx.<0] = -1
    approx = trunc(Int, approx)

    adir_train_confusion = zeros(Int, 3, 3)
    adir_test_confusion = zeros(Int, 3, 3)
    ados_train_confusion = zeros(Int, 3, 3)
    ados_test_confusion = zeros(Int, 3, 3)

    for i=1:m
        for j=adir_indices
            if train_data[i, j] != 0
                adir_train_confusion[train_data[i, j]+2, approx[i, j]+2] += 1
            end
            if test_data[i, j] != 0
                adir_test_confusion[test_data[i, j]+2, approx[i, j]+2] += 1
            end
        end
        for j=ados_indices
            if train_data[i, j] != 0
                ados_train_confusion[train_data[i, j]+2, approx[i, j]+2] += 1
            end
            if test_data[i, j] != 0
                ados_test_confusion[test_data[i, j]+2, approx[i, j]+2] += 1
            end
        end
    end
    push!(error, (adir_test_confusion[1, 3]+adir_test_confusion[3, 1]+ados_test_confusion[1, 3]+ados_test_confusion[3, 1])/(sum(adir_test_confusion)+sum(ados_test_confusion)))
    println("Error ", (adir_train_confusion[1, 3]+adir_train_confusion[3, 1]+ados_train_confusion[1, 3]+ados_train_confusion[3, 1])/(sum(adir_train_confusion)+sum(ados_train_confusion)), " ", 
                    (adir_test_confusion[1, 3]+adir_test_confusion[3, 1]+ados_test_confusion[1, 3]+ados_test_confusion[3, 1])/(sum(adir_test_confusion)+sum(ados_test_confusion)))

    push!(adir_error, (adir_test_confusion[1, 3]+adir_test_confusion[3, 1])/sum(adir_test_confusion))
    println("ADIR ", (adir_train_confusion[1, 3]+adir_train_confusion[3, 1])/sum(adir_train_confusion), " ", 
                    (adir_test_confusion[1, 3]+adir_test_confusion[3, 1])/sum(adir_test_confusion))

    push!(ados_error, (ados_test_confusion[1, 3]+ados_test_confusion[3, 1])/sum(ados_test_confusion))
    println("ADOS ", (ados_train_confusion[1, 3]+ados_train_confusion[3, 1])/sum(ados_train_confusion), " ", 
                    (ados_test_confusion[1, 3]+ados_test_confusion[3, 1])/sum(ados_test_confusion))
end

1


LoadError: BoundsError: attempt to access 0-element Array{Any,1} at index [1]

## Tune Parameters

In [98]:
using Plots
plotly() # Choose the Plotly.jl backend for web interactivity
labels = Array{String}(1, 3)
labels[1] = "All Items"
labels[2] = "ADI-R Items"
labels[3] = "ADOS Items"
plot([error, adir_error, ados_error], xticks = 0:1:20, linewidth=2, 
    title="Imputation Error as a function of k", ylabel="Validation Error", xlabel="k", label=labels,
    palette=["#ce93d8", "#ef6c00", "#4db6ac"])

#labels[1] = ""
#labels[2] = "ADI-R"
#labels[3] = "ADOS"

#plot!([[], only_adir_error, only_ados_error], linewidth=2, 
#    title="Single Instrument Imputation Error <br>as a function of k", ylabel="Validation Error", legend=:right, 
#    xlabel="k", xticks = 0:1:10, label=labels, palette = ["#ce93d8", "#4db6ac", "#ef6c00"])


## ROC

In [83]:
approx = Xs[5].'*Ys[5]

function roc(column_indices)
    i_values = approx[:, column_indices][heldout_data[:, column_indices] .!= 0]
    a_values = heldout_data[:, column_indices][heldout_data[:, column_indices] .!= 0]
    
    roc_values = sort(collect(filter(x -> (x[2] != 0 && x[1] != 0), zip(i_values, a_values))))
    tp = sum(map(x -> x[2] > 0, roc_values))
    fp = sum(map(x -> x[2] < 0, roc_values))
    tn = 0
    fn = 0
    sensitivity = Array(Float64, 0)
    specificity = Array(Float64, 0)
    cutoffs = Array(Float64, 0)
    push!(sensitivity, 1)
    push!(specificity, 0)
    push!(cutoffs, 0)
    for v=roc_values
        if v[2] > 0 
            tp -=1
            fn += 1
        elseif v[2] < 0
            fp -= 1
            tn += 1
        end
        if (abs(tp/(tp+fn) - sensitivity[end]) > .01) || (abs(tn/(tn+fp) - specificity[end]) > .01)
            push!(sensitivity, tp/(tp+fn))
            push!(specificity, tn/(tn+fp))
            push!(cutoffs, v[1])
        end
    end
    push!(sensitivity, 0)
    push!(specificity, 1)
    return sensitivity, specificity, cutoffs
end

roc (generic function with 1 method)

In [84]:
# Baseline
baseline_impute = ones(m, 1)*sum(train_data, 1)

function baseline(column_indices)
    i_values = baseline_impute[:, column_indices][heldout_data[:, column_indices] .!= 0]
    a_values = heldout_data[:, column_indices][heldout_data[:, column_indices] .!= 0]
  
    roc_values = sort(collect(filter(x -> (x[2] != 0 && x[1] != 0), zip(i_values, a_values))))
    tp = sum(map(x -> (x[1] > 0 && x[2] > 0), roc_values))
    fp = sum(map(x -> (x[1] > 0 && x[2] < 0), roc_values))
    tn = sum(map(x -> (x[1] < 0 && x[2] < 0), roc_values))
    fn = sum(map(x -> (x[1] < 0 && x[2] > 0), roc_values))

    baseline_sensitivity = tp/(tp+fn)
    baseline_specificity = tn/(tn+fp)
    return baseline_sensitivity, baseline_specificity
end

baseline (generic function with 1 method)

In [85]:
sensitivity, specificity, cutoffs = roc(1:n)
adir_sensitivity, adir_specificity, adir_cutoffs = roc(adir_indices)
ados_sensitivity, ados_specificity, ados_cutoffs = roc(ados_indices)

threshold = Int(round((findfirst(x -> x >= 0, cutoffs[2:end]) +
            findfirst(x -> x > 0, cutoffs[2:end]))/2))
adir_threshold= Int(round((findfirst(x -> x >= 0, adir_cutoffs[2:end]) +
            findfirst(x -> x > 0, adir_cutoffs[2:end]))/2))
ados_threshold = Int(round((findfirst(x -> x >= 0, ados_cutoffs[2:end]) +
            findfirst(x -> x >= 0, ados_cutoffs[2:end]))/2))

baseline_sensitivity, baseline_specificity = baseline(1:n)
adir_baseline_sensitivity, adir_baseline_specificity = baseline(adir_indices)
ados_baseline_sensitivity, ados_baseline_specificity = baseline(ados_indices)

(0.8574456809750928,0.5535309973045822)

In [86]:
using Plots
plotly() # Choose the Plotly.jl backend for web interactivity
plot(1-specificity, sensitivity, linewidth=2, 
    title="Within Instrument Imputation ROC", ylabel="Sensitivity", xlabel="1-Specificity", linewidth=2, label="GLRM Impute<br>k=5", primary=true,
    palette=[ "#ef6c00", "#695d46"], xticks = 0:.2:1, yticks = 0:.2:1, size=(500, 400), legend=:right, margin=5mm)
plot!([1-specificity[threshold]], [sensitivity[threshold]], markersize=5, markershape = :hexagon, primary=false)
plot!([1-baseline_specificity], [baseline_sensitivity], 
    markersize=5, markershape = :hexagon, primary=true, label="Median Impute")

#plot!(1-adir_specificity, adir_sensitivity, linewidth=2, 
#    linewidth=2, label="ADI-R", primary=true)
#plot!([1-adir_specificity[adir_threshold]], [adir_sensitivity[adir_threshold]], 
#    markersize=5, markershape = :hexagon, primary=false)
#plot!([1-adir_baseline_specificity], [adir_baseline_sensitivity], 
#    markersize=5, markershape = :hexagon, primary=true, label="Median ADI-R Impute")

#plot!(1-ados_specificity, ados_sensitivity, linewidth=2, 
#    linewidth=2, label="ADOS", primary=true)
#plot!([1-ados_specificity[ados_threshold]], [ados_sensitivity[ados_threshold]], 
#    markersize=5, markershape = :hexagon, primary=false)
#plot!([1-ados_baseline_specificity], [ados_baseline_sensitivity], 
#    markersize=5, markershape = :hexagon, primary=true, label="Median ADOS Impute")

In [75]:
# Accuracy

i_values = approx[heldout_data .!= 0]
a_values = heldout_data[heldout_data .!= 0]
roc_values = sort(collect(filter(x -> (x[2] != 0 && x[1] != 0), zip(i_values, a_values))))
tp = sum(map(x -> (x[2] > 0 && x[1] > 0), roc_values))
tn = sum(map(x -> (x[2] < 0 && x[1] < 0), roc_values))
println((tp+tn)/size(roc_values, 1))    

i_values = approx[:, adir_indices][heldout_data[:, adir_indices] .!= 0]
a_values = heldout_data[:, adir_indices][heldout_data[:, adir_indices] .!= 0]
roc_values = sort(collect(filter(x -> (x[2] != 0 && x[1] != 0), zip(i_values, a_values))))
tp = sum(map(x -> (x[2] > 0 && x[1] > 0), roc_values))
tn = sum(map(x -> (x[2] < 0 && x[1] < 0), roc_values))
println((tp+tn)/size(roc_values, 1)) 

i_values = approx[:, ados_indices][heldout_data[:, ados_indices] .!= 0]
a_values = heldout_data[:, ados_indices][heldout_data[:, ados_indices] .!= 0]
roc_values = sort(collect(filter(x -> (x[2] != 0 && x[1] != 0), zip(i_values, a_values))))
tp = sum(map(x -> (x[2] > 0 && x[1] > 0), roc_values))
tn = sum(map(x -> (x[2] < 0 && x[1] < 0), roc_values))
println((tp+tn)/size(roc_values, 1)) 


0.7993804956035172
0.8080629244790155
0.7801873476196587


# Output

In [17]:
k = 6
approx = Xs[k].'*Ys[k]
approx[approx.>=0] = 1
approx[approx.<0] = 0
approx = trunc(Int, approx)

new_df = convert(DataFrame, approx)
names!(new_df, names(df))
new_df = hcat(samples, new_df)
writetable("../data/impute_logloss_Z$(k).csv", new_df, separator = ',', header = true)

# Replace imputed values with real values if we have them
approx[all_data.>0] = 1
approx[all_data.<0] = 0

new_df = convert(DataFrame, approx)
names!(new_df, names(df))
new_df = hcat(samples, new_df)
writetable("../data/impute_logloss_realfill_Z$(k).csv", new_df, separator = ',', header = true)

X_df = convert(DataFrame, Xs[k].')
X_df = hcat(samples, X_df)
writetable("../data/impute_logloss_X$(k).csv", X_df, separator = ',', header=false)

Y_df = convert(DataFrame, Ys[k])
print(size(names(df)), size(Y_df))
names!(Y_df, names(df))
writetable("../data/impute_logloss_Y$(k).csv", Y_df, separator = ',', header=true)

(123,)(6,123)

In [65]:

k = 6
approx = adir_Xs[k].'*adir_Ys[k]
approx[approx.>=0] = 1
approx[approx.<0] = 0
approx = trunc(Int, approx)

# Replace imputed values with real values if we have them
approx[all_data[:, adir_indices].>0] = 1
approx[all_data[:, adir_indices].<0] = 0

new_df = convert(DataFrame, approx)
names!(new_df, names(df)[1:size(approx, 2)])
new_df = hcat(samples, new_df)

writecsv("../data/impute_logloss_adir_realfill_X$(k).csv", adir_Xs[k])
writecsv("../data/impute_logloss_adir_realfill_Y$(k).csv", adir_Ys[k])
writetable("../data/impute_logloss_adir_realfill_Z$(k).csv", new_df, separator = ',', header = true)

In [1]:
k = 6
approx = ados_Xs[k].'*ados_Ys[k]
approx[approx.>=0] = 1
approx[approx.<0] = 0
approx = trunc(Int, approx)

# Replace imputed values with real values if we have them
approx[all_data[:, ados_indices].>0] = 1
approx[all_data[:, ados_indices].<0] = 0

new_df = convert(DataFrame, approx)
names!(new_df, names(df)[(end-size(approx, 2)+1):end])
new_df = hcat(samples, new_df)

writecsv("../data/impute_logloss_ados_realfill_X$(k).csv", adir_Xs[k])
writecsv("../data/impute_logloss_ados_realfill_Y$(k).csv", adir_Ys[k])
writetable("../data/impute_logloss_ados_realfill_Z$(k).csv", new_df, separator = ',', header = true)

LoadError: UndefVarError: ados_Xs not defined